## Определение языка и перевод (+ обработка данных об изготовителе)

На этом этапе:
- Используем `langdetect` для определения языка;
- Применяем `deep_translator` для перевода на английский;
- Пропускаем перевод, если текст уже на английском;
- Логируем необработанные случаи.


In [ ]:
!pip install deep-translator langdetect --quiet

import json
import sqlite3
import re
from deep_translator import GoogleTranslator
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
import pandas as pd

# Стабильность результатов langdetect
DetectorFactory.seed = 42

In [ ]:
def is_non_empty(text):
    return bool(text and text.strip())

def contains_cyrillic(text):
    return bool(re.search(r'[а-яА-Я]', text))

def detect_language(text):
    try:
        return detect(text)
    except LangDetectException:
        return "unknown"

def translate_to_english(text):
    if not text or text.strip() == "":
        return ""

    lang = detect_language(text)

    if lang == "en" and contains_cyrillic(text):
        force_translate = True
    else:
        force_translate = False

    if lang == "en" and not force_translate:
        return text
    elif lang == "unknown":
        print ("unkown", text)
        return text

    try:
        print(text,"\n")
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        print(translated,"\n")
        return translated
    except Exception as e:
        print(f"Translation failed: '{text[:50]}...' — {e}")
        return text

In [ ]:
#попытка разделить данные об изготовителе по тегам
def process_info(info_text):
    info_parts = info_text.split('<br><br>')

    info_dict = {
        "страна происхождения": "",
        "изготовитель": "",
        "ОГРН": "",
        "юридический адрес": "",
        "продавец": "",
        "info": ""
    }

    if len(info_parts) > 0:
        # Разбираем первую часть, которая содержит "страна происхождения"
        country_info = info_parts[0].strip()
        if '<br>' in country_info:
            country_parts = country_info.split('<br>', 1)
            info_dict["страна происхождения"] = country_parts[1].strip()

    if len(info_parts) > 1:
        # Обрабатываем "изготовитель"
        manufacturer_info = info_parts[1].strip()
        if '<br>' in manufacturer_info:
            manufacturer_parts = manufacturer_info.split('<br>', 1)
            info_dict["изготовитель"] = manufacturer_parts[1].strip()

    if len(info_parts) > 2:
        # Обрабатываем "продавец"
        seller_info = info_parts[2].strip()
        if '<br>' in seller_info:
            seller_parts = seller_info.split('<br>', 1)
            if len(seller_parts) > 1:
                info_dict["продавец"] = seller_parts[1].strip()

    if len(info_parts) > 3:
        # Обрабатываем "ОГРН"
        ogrn_info = info_parts[3].strip()
        if '<br>' in ogrn_info:
            ogrn_parts = ogrn_info.split('<br>', 1)
            info_dict["ОГРН"] = ogrn_parts[1].strip()

    if len(info_parts) > 4:
        # Обрабатываем "юридический адрес"
        address_info = info_parts[4].strip()
        if '<br>' in address_info:
            address_parts = address_info.split('<br>', 1)
            info_dict["юридический адрес"] = address_parts[1].strip()

    # Остальная информация записывается в "info"
    if len(info_parts) > 5:
        info_dict["info"] = ' '.join(info_parts[5:]).strip()

    return info_dict

In [ ]:
def process_composition(composition):
    if not is_non_empty(composition):
        return ""

    composition = translate_to_english(composition)
    composition = composition.lower()

    return composition


def process_products(json_data):
    processed_data = []

    for index, product in enumerate(json_data):
        #print(f"Обработка продукта {index + 1} из {len(json_data)}: {product.get('itemId', 'Неизвестный itemId')}")

        product_info = product.get('info информация', '')

        # Обработка composition
        composition = product.get('composition', '')
        composition = process_composition(composition)

        # Обработка информации
        info_processed = process_info(product_info)

        processed_data.append({
            "itemId": product.get('itemId', ''),
            "name": product.get('name', ''),
            "type": product.get('type', ''),
            "brand": product.get('brand', ''),
            "productType": product.get('productType', ''),
            "composition": composition,
            "description": product.get('description', ''),
            "страна происхождения": info_processed["страна происхождения"],
            "изготовитель": info_processed["изготовитель"],
            "продавец": info_processed["продавец"],
            "ОГРН": info_processed["ОГРН"],
            "юридический адрес": info_processed["юридический адрес"],
            "info": info_processed["info"]
        })

    return processed_data

def save_to_db(processed_data):
    conn = sqlite3.connect('full_products_database.db')
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS merged_table (
                    itemId TEXT,
                    name TEXT,
                    type TEXT,
                    brand TEXT,
                    productType TEXT,
                    composition TEXT,
                    description TEXT,
                    страна_происхождения TEXT,
                    изготовитель TEXT,
                    продавец TEXT,
                    ОГРН TEXT,
                    юридический_адрес TEXT,
                    info TEXT
                )''')

    for product in processed_data:
        c.execute('''INSERT INTO merged_table (
                        itemId, name, type, brand, productType, composition, description,
                        страна_происхождения, изготовитель, продавец, ОГРН, юридический_адрес, info
                    ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', (
                        product['itemId'],
                        product['name'],
                        product['type'],
                        product['brand'],
                        product['productType'],
                        product['composition'],
                        product['description'],
                        product['страна происхождения'],
                        product['изготовитель'],
                        product['продавец'],
                        product['ОГРН'],
                        product['юридический адрес'],
                        product['info']
                    ))

    conn.commit()
    conn.close()

In [ ]:
with open('merged_data.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)

processed_data = process_products(json_data)
save_to_db(processed_data)

Выходные данные были обрезаны до нескольких последних строк (5000).
Exfoliant: water, propylene glycol, gluconolactone, glycerin, lactobionic acid, glycolic acid, dimethylaminoetanol (DME), citric acid, sodium lauryl sulfate, grape extract (vitis vinifere), carboximethyl cellulose, butyleneglycol, dinatriy Edta. <br> <br>

Concentrate: water, propylene glycol, glycerin, aloe Barbado juice (ALOE BARBADensis), enzymatic lizat bifidobacteria, melatonin, gyaluronate of sifty, sheep tumber (albatrellus ovinus), sodium ascorbilphosphate, priestolus, dairy Acid, dynatrihosphate, dynatri Edta, sodium phosphate, phenoxyethanol, citric acid, sodium benzoate, potassium sorbate. 

Aqua/[water]*, Pentylene Glycol, Methylpropanediol, Caprylyl Glycol, Acrylates/c10-30 Alkyl Acrylate Crosspolymer, Sodium Hydroxide, Sodium Citrate, Polysorbate 20, Phenylpropanol, Lactose, Parfum/[fragrance], Whey Protein, Lactic Acid, Sodium Benzoate, Citric Acid.
<br><br>
*Соntains Swiss Alpine Glacier Water. 

Aqua/[

## Очистка и стандартизация текста

На этом этапе:
- Приводим текст к чистому и структурированному виду;
- Удаляем HTML-теги, лишние пробелы и пунктуацию;
- Приводим текст к нижнему регистру;
- Применяем токенизацию и лемматизацию для составов и описаний;
- Сохраняем важные конструкции (например, отрицания).


Обработаем описания средств. Удалим стоп-слова, но оставим нужные,
например, отрицания - у них есть смысловая нагрузка

In [ ]:
!pip install spacy


In [ ]:
!python -m spacy download ru_core_news_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 56.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import sqlite3
import re
import spacy

In [ ]:
nlp = spacy.load('ru_core_news_sm')

In [ ]:
important_stopwords = {'не', 'нет', 'без', 'против', 'для', 'в', 'на'}

def preprocess_text(text):
    text = re.sub(r'<.*?>', '', text)

    text = " ".join(text.split())

    text = text.lower()

    text = re.sub(r'[^\w\s]', '', text)

    return text

def tokenize_text_russian(text):
    if text:
      text = preprocess_text(text)
      doc = nlp(text)
      tokens = [
            token.lemma_ for token in doc
            if not token.is_stop or token.text in important_stopwords
            and not token.is_punct
        ]
      return ' '.join(tokens)
    return ""

In [ ]:
def tokenize_and_update_database(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("PRAGMA table_info(merged_table);")
    columns = [column[1] for column in cursor.fetchall()]

    if 'description_tokens' not in columns:
        cursor.execute('''
            ALTER TABLE merged_table
            ADD COLUMN description_tokens TEXT;
        ''')
        conn.commit()

    cursor.execute("SELECT itemId, description FROM merged_table")
    rows = cursor.fetchall()

    # Токенизируем каждое описание и записываем результат в новую колонку
    for row in rows:
        itemId, description = row
        description_tokens = tokenize_text_russian(description)

        # Обновляем запись в базе данных
        cursor.execute('''
            UPDATE merged_table
            SET description_tokens = ?
            WHERE itemId = ?
        ''', (description_tokens, itemId))

    conn.commit()
    conn.close()


In [ ]:
db_path = '/content/full_products_database.db'

tokenize_and_update_database(db_path)

print("Токенизация и обновление базы данных завершены.")

Токенизация и обновление базы данных завершены.


Теперь обработаем составы:

In [1]:
!pip install scispacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 82.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.3/243.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.2/920.2 kB 37.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 110.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: blis
    Found existing installation: blis 1.3.0
    Uninstalling blis-1.3.0:
      Successfully uninstalled blis-1

In [2]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_sm-0.5.4.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 22.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for en_core_sci_sm: filename=en_core_sci_sm-0.5.4-py3-none-any.whl size=14778488 sha256=fd58d5a459a7da1c4b7b07759f389661ff28c32eb17b1f6c7d32e0ac946bb0f8
  Stored in directory: /root/.cache/pip/wheels/7f/29/44/dd461872b8547b8e8007f03418fb8061f5c05c71447982bcff
Successfully built en_core_sci_sm


In [3]:
!pip install spacy==3.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: typer
    Found existing installation: typer 0.15.3
    Uninstalling typer-0.15.3:
      Successfully uninstalled typer-0.15.3
  Attempting uninstall: smart-open
    Found existing installation: smart-open 7.1.0
    Uninstalling smart-open-7.1.0:
      Successfully uninstalled smart-open-7.1.0
  Attempting uninstall: cloudpathlib
    Found existing installation: cloudpathlib 0.21.0
    Uninstalling cloudpathlib-0.21.0:
      Successfully uninstalled cloudpathlib-0.21.0
  Attempting uninstall: weasel
    Found existing installation: weasel 0.4.1
    Uninstalling weasel-0.4.1:
     

In [ ]:
import sqlite3
import spacy
import scispacy
import re

nlp = spacy.load("en_core_sci_sm")

In [ ]:
import requests
import time
import json

In [ ]:
with open('chemical_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

components_with_404 = set(data.get("components_with_404", [])) # Множество для отслеживания химических веществ с ошибкой 404
synonyms_cache = data.get("synonyms_cache", {}) # Словарь для хранения синонимов химических веществ

def get_pubchem_synonyms(chemical_name, retries=7, delay=4, timeout=10):
    # Проверяем, если вещество уже есть в списке с ошибкой 404 или в кэше
    if chemical_name in components_with_404:
        print(f"Компонент {chemical_name} уже был с 404, пропускаем.")
        return []

    if chemical_name in synonyms_cache:
        print(f"Синонимы для {chemical_name} найдены в кэше.")
        return synonyms_cache[chemical_name]

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{chemical_name}/synonyms/JSON"

    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=timeout)

            if response.status_code == 404:
                print(f"404 для компонента {chemical_name} по ссылке {url} - добавляю в список ошибок")
                components_with_404.add(chemical_name)
                synonyms_cache[chemical_name] = []  # Сохраняем пустой список для компонента с 404
                return []

            if response.status_code == 503:
                print(f"503 для компонента {chemical_name} по ссылке {url} - сервер временно недоступен, ожидаем...")
                time.sleep(delay)
                delay *= 2
                continue

            response.raise_for_status()
            data = response.json()
            print(data)

            # Проверяем, есть ли в данных синонимы
            if "InformationList" in data and "Information" in data["InformationList"]:
                synonyms_info = data["InformationList"]["Information"][0]
                if 'Synonym' in synonyms_info:
                    synonyms = synonyms_info['Synonym']
                    synonyms_cache[chemical_name] = synonyms  # Сохраняем синонимы в кэш
                    return synonyms
                else:
                    print(f"Синонимы для {chemical_name} не найдены.")
                    components_with_404.add(chemical_name)  # Добавляем в список для пропуска в будущем
                    synonyms_cache[chemical_name] = []  # Сохраняем пустой список для отсутствующих синонимов
                    return []  # Нет синонимов, возвращаем пустой список
            else:
                print(f"Информация о синонимах для {chemical_name} отсутствует.")
                components_with_404.add(chemical_name)  # Добавляем в список для пропуска в будущем
                synonyms_cache[chemical_name] = []  # Сохраняем пустой список для отсутствующей информации
                return []

        except requests.exceptions.RequestException as e:
            print(f"Ошибка при получении данных для {chemical_name}: {e}")
            if attempt < retries - 1:
                print(f"Retrying... ({attempt + 1}/{retries})")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"Не удалось получить данные для {chemical_name} после {retries} попыток.")
                components_with_404.add(chemical_name)  # Добавляем в список для пропуска в будущем
                synonyms_cache[chemical_name] = []  # Сохраняем пустой список для неудачного запроса
                return []

def save_data():
    with open('synonyms_data.json', 'w', encoding='utf-8') as f:
        json.dump({
            "components_with_404": list(components_with_404),
            "synonyms_cache": synonyms_cache
        }, f, ensure_ascii=False, indent=2)


In [ ]:
def normalize_to_preferred_synonym(chemical_name):
    synonyms = get_pubchem_synonyms(chemical_name)
    if synonyms:
        # Используем первый синоним как предпочтительный
        preferred_synonym = synonyms[0].lower()
        return preferred_synonym
    else:
        return chemical_name.lower()

In [ ]:
def normalize_tokens(doc):
    normalized_tokens = []
    previous_token = None

    for token in doc:
        if (token.pos_ != 'PUNCT' and
        not token.is_stop and
        token.text.strip() != '' and
        token.text.lower() not in ['leaf', 'vitamin', 'caviar', 'cola', 'line', 'lady', 'freeze', 'snow']): #для этих слов неправильные синонимы
            normalized_name = normalize_to_preferred_synonym(token.text)  # Синхронный вызов

            # Если новый нормализованный токен отличается от предыдущего, добавляем его
            if normalized_name != previous_token:
                normalized_tokens.append(normalized_name)

            previous_token = normalized_name
        else:
            normalized_tokens.append(token.text)

    return normalized_tokens


In [ ]:
def join_tokens_into_phrases(tokens):
    phrases = []
    current_phrase = []

    for token in tokens:
        if token not in {',', '.', ';', '•', '･'}:
            current_phrase.append(token)
        else:
            if current_phrase:
                phrases.append(" ".join(current_phrase))
                current_phrase = []
            phrases.append(';')

    if current_phrase:
        phrases.append(" ".join(current_phrase))

    return phrases

In [ ]:
import sqlite3

conn = sqlite3.connect('/content/full_products_database.db')
cursor = conn.cursor()

# добавляем новую колонку
cursor.execute('''
    ALTER TABLE merged_table ADD COLUMN composition_tokenized TEXT
''')
conn.commit()

# очищаем текст
def process_composition_text(composition_text):
    composition_text = re.sub(r'[()\/]', ' ', composition_text)
    doc = nlp(composition_text)

    normalized_tokens = normalize_tokens(doc)
    phrases = join_tokens_into_phrases(normalized_tokens)

    return ''.join(phrases)

def process_database():
    cursor.execute('SELECT itemId, composition FROM merged_table')
    rows = cursor.fetchall()

    product_counter = 1

    for item_id, composition in rows:
        print(f"Токенизация состава для продукта #{product_counter} (ID: {item_id})...")
        if composition:
            tokenized_composition = process_composition_text(composition)

            cursor.execute('''
                UPDATE merged_table
                SET composition_tokenized = ?
                WHERE itemId = ?
            ''', (tokenized_composition, item_id))
        product_counter += 1

    conn.commit()
    conn.close()

process_database()
save_data()


Выходные данные были обрезаны до нескольких последних строк (5000).
Синонимы для polysorbate найдены в кэше.
Синонимы для 20 найдены в кэше.
Синонимы для caprylic найдены в кэше.
Синонимы для capric найдены в кэше.
Синонимы для triglyceride найдены в кэше.
Синонимы для arginine найдены в кэше.
Синонимы для dipotassium найдены в кэше.
Синонимы для glycyrrhizate найдены в кэше.
Синонимы для hydroxyacetophenone найдены в кэше.
Синонимы для 1,2-hexanediol найдены в кэше.
Синонимы для ethylhexylglycerin найдены в кэше.
Синонимы для disodium найдены в кэше.
Синонимы для edta найдены в кэше.
Токенизация состава для продукта #4206 (ID: 19000187279)...
Синонимы для water найдены в кэше.
Синонимы для glycerin найдены в кэше.
Синонимы для propanediol найдены в кэше.
Синонимы для butylene найдены в кэше.
Синонимы для glycol найдены в кэше.
Синонимы для 1,2-hexanediol найдены в кэше.
Синонимы для niacinamide найдены в кэше.
Синонимы для prunus найдены в кэше.
Синонимы для cerasus найдены в кэше.
Си

## Чтение таблицы сочетаний и конфликтов компонентов

* С помощью `pdfplumber` извлекается таблица из PDF-файла (например, `Таблица сочетаний и конфликтов ART&FACT_.pdf` и другие).
* Формируется `DataFrame`, где:
   - строки и столбцы представляют различные активные ингредиенты (например, Retinol, Niacinamide),
   - а значения на пересечении — это описание их совместимости или предупреждения (например, "не рекомендовано", "можно").
* Далее для каждого сочетания из таблицы все возможные пары строка × столбец (с учетом синонимов) добавляются в таблицу в SQLite-базе.


In [ ]:
!pip install pdfplumber
import pdfplumber

In [ ]:
with pdfplumber.open('/content/Таблица сочетаний и конфликтов ART\&FACT\_.pdf') as pdf:
page = pdf.pages[0]

table = page.extract_table()

for row in table:
print(row)

In [ ]:
import sqlite3
import pandas as pd

In [ ]:
# списки компонентов для колонн и рядов - активные действующие компоненты из таблицы
columns = [
'Niacinamide 2–5 %',
'Niacinamide > 5 %, nicotinamide',
'Vitamin C Ascorbic Acid ≤ 5 %, Ascorbic Acid',
'Vitamin C Ascorbic Acid > 5 % рН ≤ 4, Ascorbic Acid',
'Vitamin C, Ascorbyl Glucoside, Magnesium Ascorbyl Phosphate, 3-O-Ethyl Ascorbic Acid, Sodium Ascorbyl Phosphate, Tetrahexyldecyl Ascorbate',
'Blue Retinol Bakuchiol',
'Retinol, Retinol Acetate, Retinol Palmitate, Retinal, Retinyl Propionate, Retinol Retinoate',
'Retinol encapsulated and liposomal form',
'Azelaic Acid',
'Amino Acids + Peptides,  EGF, Matrixyl, Leuphasyl, Argireline, SYN-AKE',
'ВНА-acids,  Salicylic Acid 0.5–2 % pH = 3–4, salicylic acid, salicylic',
'AНА-acids, Lactic Acid, Glycolic Acid, Mandelic Acid 2–10 %\* рН = 3–4, cyclandelate',
'РНА-acids,  Lactobionic Acid, Gluconolactone 2–10 % pH = 3–4, Gluconolactone',
'Benzoyl Peroxide 2.5–5 %, Benzoyl Peroxide'
]

rows = [
'Niacinamide 2–5 %',
'Niacinamide > 5 %, nicotinamide',
'Vitamin C Ascorbic Acid ≤ 5 %, Ascorbic Acid',
'Vitamin C Ascorbic Acid > 5 % рН ≤ 4, Ascorbic Acid',
'Vitamin C, Ascorbyl Glucoside, Magnesium Ascorbyl Phosphate, 3-O-Ethyl Ascorbic Acid, Sodium Ascorbyl Phosphate, Tetrahexyldecyl Ascorbate',
'Blue Retinol Bakuchiol',
'Retinol, Retinol Acetate, Retinol Palmitate, Retinal, Retinyl Propionate, Retinol Retinoate',
'Retinol encapsulated and liposomal form',
'Azelaic Acid',
'Amino Acids + Peptides,  EGF, Matrixyl, Leuphasyl, Argireline, SYN-AKE',
'ВНА-acids,  Salicylic Acid 0.5–2 % pH = 3–4, salicylic acid, salicylic',
'AНА-acids, Lactic Acid, Glycolic Acid, Mandelic Acid 2–10 %\* рН = 3–4, cyclandelate',
'РНА-acids,  Lactobionic Acid, Gluconolactone 2–10 % pH = 3–4, Gluconolactone',
'Benzoyl Peroxide 2.5–5 %, Benzoyl Peroxide'
]

df = pd.DataFrame(table, columns=columns, index=rows)

# Подключение к базе данных

conn = sqlite3.connect('components-combo-3.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS components (
id INTEGER PRIMARY KEY,
first_element TEXT,
second_element TEXT,
result TEXT,
comment TEXT
)''')

for i in range(len(df.index)):
  for j in range(i, len(df.columns)):
    row = df.index[i]
    col = df.columns[j]
    comment = df.at[row, col]

    # Разделяем компоненты по запятой и обрабатываем их по одному
    row_elements = [e.strip() for e in row.split(',')]
    col_elements = [e.strip() for e in col.split(',')]

    # Вставляем в базу для каждого элемента
    for r in row_elements:
        for c in col_elements:
            cursor.execute('''INSERT INTO components (first_element, second_element, result, comment)
                            VALUES (?, ?, ?, ?)''', (r, c, '', comment))

conn.commit()
conn.close()

print("Данные успешно сохранены в базу данных.")

## Проверка наличия ключевых компонентов в составе продукта

На этом этапе мы проверяем, содержит ли продукт один или несколько важных активных компонентов (формирование векторов характеристик), которые могут участвовать в сочетаниях или конфликтах (из предыдущего шага).
* Каждый состав (`composition_tokenized`) разбивается по `;`, чтобы получить отдельные компоненты.
* Для каждого элемента из списка `flags`:
   - Ищем его точное или близкое совпадение среди токенов состава.
   - Сохраняем результат как:
     - `1` — есть точное совпадение,
     - `0` — есть только похожие совпадения,
     - `-1` — не найдено ничего подходящего.
* В таблицу `products` добавляются новые столбцы по каждому компоненту из `flags` (названия колонок автоматически очищаются и приводятся к нижнему регистру), флаги сохраняются в базу.


In [ ]:
import re

#функция нормализации названий компонентов, чтобы можно было назвать столбец
def normalize_ingredient_name(name):
    name = name.strip()

    name = name.replace('%', 'pct')
    name = re.sub(r'[^\w\s\-]', '', name)
    name = name.replace(' ', '_').replace('-', '_')

    # если начинается с цифры — добавим префикс (нельзя начинать имя поля в SQLite с цифры)
    if re.match(r'^\d', name):
        name = f"i_{name}"

    name = name.lower()

    return name


In [ ]:
#создадим отдельную бд, где будут только itemId, composition_tokenized и вектора характеристик
import sqlite3

source_conn = sqlite3.connect('/content/full_products_database.db')
source_cursor = source_conn.cursor()

# Создаём новую БД
new_conn = sqlite3.connect('/content/composition_vectors.db')
new_cursor = new_conn.cursor()

new_cursor.execute('''
    CREATE TABLE IF NOT EXISTS tokenized_table (
        itemId TEXT,
        composition_tokenized TEXT
    )
''')

# Читаем данные из старой БД
try:
    source_cursor.execute("SELECT itemId, composition_tokenized FROM merged_table")
    rows = source_cursor.fetchall()
except sqlite3.DatabaseError as e:
    print("Ошибка при чтении из БД:", e)
    source_conn.close()
    new_conn.close()
    raise

# Записываем данные в новую
new_cursor.executemany('''
    INSERT INTO tokenized_table (itemId, composition_tokenized)
    VALUES (?, ?)
''', rows)

new_conn.commit()

source_conn.close()
new_conn.close()

print("Данные успешно скопированы.")


In [ ]:
#активные компоненты, которые будут искаться

flags = [
    'Niacinamide 2–5 %',
    'Niacinamide > 5 %',
    'nicotinamide',
    'Vitamin C Ascorbic Acid ≤ 5 %',
    'Ascorbic Acid',
    'Vitamin C Ascorbic Acid > 5 % рН ≤ 4',
    'Vitamin C',
    'Ascorbyl Glucoside',
    'Magnesium Ascorbyl Phosphate',
    '3-O-Ethyl Ascorbic Acid',
    'Sodium Ascorbyl Phosphate',
    'Tetrahexyldecyl Ascorbate',
    'Blue Retinol Bakuchiol',
    'Retinol',
    'Retinol Acetate',
    'Retinol Palmitate',
    'Retinal',
    'Retinyl Propionate',
    'Retinol Retinoate',
    'Retinol encapsulated and liposomal form',
    'Azelaic Acid',
    'Amino Acids + Peptides',
    'EGF',
    'Matrixyl',
    'Leuphasyl',
    'Argireline',
    'SYN-AKE',
    'ВНА-acids',
    'Salicylic Acid 0.5–2 % pH = 3–4',
    'salicylic acid',
    'salicylic',
    'AНА-acids',
    'Lactic Acid',
    'Glycolic Acid',
    'Mandelic Acid 2–10 %* рН = 3–4',
    'cyclandelate',
    'РНА-acids',
    'Lactobionic Acid',
    'Gluconolactone 2–10 % pH = 3–4',
    'Gluconolactone',
    'Benzoyl Peroxide 2.5–5 %',
    'Benzoyl Peroxide'
]

In [ ]:
import sqlite3
import difflib
import re

# Функция для токенизации текста по символу ';'
def preprocess_text_by_semicolon(text):
    if text is None:
        return []  # Если текст None, возвращаем пустой список
    tokens = text.split(';')
    tokens = [token.strip().lower() for token in tokens]  # Очищаем от лишних пробелов и делаем строчными
    return tokens

# Функция для нахождения схожих слов
def find_similar_words(word, word_list, n=3, cutoff=0.6):
    return difflib.get_close_matches(word, word_list, n=n, cutoff=cutoff)

# Функция для проверки наличия компонентов и проставления флагов
def check_flags(compozition_text, flags):
    tokens = preprocess_text_by_semicolon(compozition_text)

    flag_results = {}

    # Для каждого компонента из списка флагов ищем его в токенах состава
    for flag in flags:
        similar_tokens = find_similar_words(flag, tokens)
        if flag.lower() in similar_tokens:
            flag_results[flag] = 1  # Есть точное или похожее совпадение
        elif len(similar_tokens) > 0:
            flag_results[flag] = 0  # Есть схожие слова, но не точные
        else:
            flag_results[flag] = -1  # Нет совпадений

    return flag_results

# Соединение с базой данных
conn = sqlite3.connect('/content/composition_vectors.db')
cursor = conn.cursor()

cursor.execute('SELECT itemId, composition_tokenized FROM tokenized_table')
rows = cursor.fetchall()

def column_exists(cursor, column_name):
    cursor.execute(f"PRAGMA table_info(tokenized_table)")  # Получаем информацию о столбцах
    columns = cursor.fetchall()
    return any(col[1] == column_name for col in columns)  # Проверяем, есть ли колонка

# Проставляем флаги для каждого компонента
for row in rows:
    product_id = row[0]
    compozition_text = row[1]

    # Получаем флаги для каждого компонента
    flag_results = check_flags(compozition_text, flags)

    # Обновляем базу данных новыми колонками с флагами
    for flag, flag_value in flag_results.items():
        column_name = normalize_ingredient_name(flag) # Создаем имя колонки

        if not column_exists(cursor, column_name):
            cursor.execute(f"ALTER TABLE tokenized_table ADD COLUMN {column_name} INTEGER")

        cursor.execute(f"UPDATE tokenized_table SET {column_name} = ? WHERE itemId = ?", (flag_value, product_id))  # Обновляем флаг для конкретного продукта

# Сохраняем изменения
conn.commit()
conn.close()


In [ ]:
# Список запрещенных компонентов
banned_ingredients = [
    "Benzene",
    "Formaldehyde",
    "Chloroform",
    "Methanol",
    "1,2-Dichloroethane",
    "Nitrosamines",
    "Acetaldehyde",
    "Bis(2-ethylhexyl) phthalate (DEHP)",
    "Dibutyl phthalate (DBP)",
    "Diisobutyl phthalate (DIBP)",
    "Toluene",
    "DDT (Dichlorodiphenyltrichloroethane)",
    "Hexachlorobenzene",
    "Lindane",
    "CI 12150",
    "CI 42555",
    "PABA (Para-aminobenzoic acid)",
    "4-Methylbenzylidene camphor (4-MBC)",
    "Hydroquinone",
    "Cresols",
    "Musk ketone",
    "Musk xylene"
]

# прогоним предыдущий кусок с этим списком, чтобы найти запрещенные компоненты


## Автоматическое определение характеристик из описаний продуктов

В этом шаге извлекаем только `itemId` и `description_tokenized` из общей базы данных, чтобы автоматически определить ключевые косметические свойства продуктов, не опираясь на состав. Цель — классифицировать продукты по назначению: например, увлажняющий, антивозрастной, от акне, для чувствительной кожи и т.д.

- создан список характеристик и синонимичных ключевых слов.
- для каждой характеристики флаг `1` (найдено) или `0` (не найдено).
- результаты в новой SQLite-базе `description_vectors.db` с таблицей `description_flags`, где каждая строка соответствует продукту и включает бинарные признаки.


In [ ]:
#выделяем itemId и description_tokens в отдельную бд
import sqlite3

source_conn = sqlite3.connect('/content/full_products_database.db')
source_cursor = source_conn.cursor()

target_conn = sqlite3.connect('/content/description_features.db')
target_cursor = target_conn.cursor()

target_cursor.execute('''
    CREATE TABLE IF NOT EXISTS description_flags (
        itemId INTEGER PRIMARY KEY,
        description_tokens TEXT
    )
''')

source_cursor.execute('SELECT itemId, description_tokens FROM merged_table')
rows = source_cursor.fetchall()

for row in rows:
    target_cursor.execute('INSERT OR REPLACE INTO description_flags (itemId, description_tokens) VALUES (?, ?)', row)

target_conn.commit()
source_conn.close()
target_conn.close()


In [ ]:
# Характеристики и ключевые слова, которые будут искаться среди токенов описаний
features = {
    "увлажняющий": ["увлажн", "влаг", "гидра", "hydrat", "moistur", "сухост", "сушит", "дегидра"],
    "антивозрастной": ["антивозраст", "anti-age", "antiaging", "возрастн", "морщин", "wrinkle", "омолож", "эластичн", "подтянут", "лифтинг", "лифт", "тонус", "зрелая", "зрелую"],
    "выравнивает_тон": ["тон кож", "выравниван", "ровн", "оттенк", "pigment", "гиперпигмент"],
    "питательный": ["питани", "питает", "насыща", "насыщен", "наполн", "восстанов", "vitamin", "витамин", "mineral", "минерал"],
    "для_чувствительной": ["чувствительн", "сенситив", "sensitive", "успокаив", "раздражен", "деликатн", "покраснен", "краснот", "розацеа", "купероз", "реактивн"],
    "для_жирной": ["жирн", "блеск", "матир", "себум", "sebum", "жирность", "shine", "oil", "маслян", "пори", "поры"],
    "от_акне": ["акне", "acne", "воспален", "высыпан", "прыщ", "угр", "comedon", "пост-акне", "постакне"],
    "для_проблемной": ["проблемн", "недостатк", "imperfect", "problem", "несовершенств"],
    "для_сухой": ["сух", "шелуш", "сушит", "шелушен", "flak", "peel", "сухост", "обезвожен", "dehydrat"],
    "для_комбинированной": ["комбинир", "combination", "смешан", "t-zone", "т-зон"],
    "можно_поверх_макияжа": ["поверх", "макияж", "makeup", "over makeup", "фиксац", "fixing"],
    "от_черных_точек": ["черные точки", "комедон", "сальные пробки", "очищение пор", "закупорка пор", "blackhead", "comedone", "pore cleansing"],
    "сияние_кожи": ["сияние", "блеск", "свечение", "glow"],

    "регенерирующий": ["регенер", "regener", "восстан", "воссоздан", "обновлен", "renew", "заживлен", "heal"],
    "матирующий": ["матир", "matte", "антиблеск", "блеск", "shine", "контроль себум"],
    "от_отеков": ["отек", "отечност", "puff", "мешк", "припухлост", "разгон лимф"],
    "для_обезвоженной": ["обезвож", "dehydrat", "водн баланс", "нехватк вод", "сухост"],
    "для_нормальной": ["нормальн", "normal", "сбалансир", "balance"],
    "spf_защита": ["spf", "солнцезащит", "sun protect", "uv", "ультрафиолет", "загар", "tan"],
    "энзимный": ["энзим", "enzyme", "фермент", "биоактив"],
    "уменьшает_поры": ["пор", "поры", "pore", "сужает пор", "чистые пор", "расширен пор"],
    "веганский": ["веган", "vegan", "растительн", "plant", "не тестир", "cruelty free"],
    "гипоаллергенный": ["гипоаллерген", "hypoallergen", "безопасн", "без раздражен"],
    "некомедогенный": ["некомедоген", "non-comedogen", "не закупор", "без закупор", "не блокирует пор"],
    "с_маслами": ["масло", "oil", "арган", "argan", "жожоба", "jojoba", "ши", "shea", "кокос", "coconut", "миндал", "almond"],
    "с_экстрактами": ["экстракт", "extract", "ромашк", "chamomile"],
    "минеральный": ["минерал", "mineral", "минеральн"],
    "органический": ["органическ", "organic", "био", "bio", "эко", "eco", "natural", "натуральн"],
    "для_подростков": ["подростк", "teen", "юнош", "молод", "young"],
    "ночной_уход": ["ночн", "night", "ночь", "сон", "sleep", "вечерн"],
    "дневной_уход": ["дневн", "day", "день", "утрен", "morning"],
    "успокаивающий": ["успок", "sooth", "calm", "peace", "мягк", "soft"],
    "очищающий": ["очищен", "clean", "purif", "детокс", "detox", "чист", "pure"],
    "отшелушивающий": ["отшелуш", "exfol", "скраб", "scrub", "пилинг", "peel"],
    "противовоспалительный": ["противовоспал", "anti-inflamm", "воспал", "inflamm", "покраснен", "redness"],
    "ароматизированный": ["аромат", "fragran", "запах", "smell", "парфюм", "perfum"],
    "неароматизированный": ["без запах", "no fragran", "без аромат", "без парфюм"],
    "гидрофильный": ["гидрофильн", "hydrophil", "водн", "water", "масло раств", "oil soluble"],
    "нежирный": ["нежирн", "non-greasy", "легк", "light", "быстро впитыва"],
    "плотный": ["плотн", "тяжел", "heavy", "густ", "thick"],
    "легкий": ["легк", "light", "невесом", "weightless", "воздушн", "airy"]
}

In [ ]:
import sqlite3
import difflib


# Подключение к базе
db_path = "/content/description_features.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Модификаторы
NEGATIONS = {"не", "без", "нет"}
SUPPORTS = {"для", "против"}

# Добавляем колонку, если её нет
cursor.execute("PRAGMA table_info(description_flags);")
existing_columns = {col[1] for col in cursor.fetchall()}

for feature in features:
    if feature not in existing_columns:
        cursor.execute(f"ALTER TABLE description_flags ADD COLUMN {feature} INTEGER DEFAULT 0;")

# Обработка описания
def process_description(text, features):
    tokens = text.lower().split()
    result = {}

    for feature, keywords in features.items():
        found = False
        suppressed = False

        for i, token in enumerate(tokens):
            prev = tokens[i - 1] if i > 0 else ""

            matches = difflib.get_close_matches(token, keywords, n=1, cutoff=0.7)
            if matches:
                if prev in NEGATIONS:
                    suppressed = True
                    break
                found = True

        result[feature] = 1 if found and not suppressed else 0

    return result

# Получаем данные и обновляем
cursor.execute("SELECT itemId, description_tokens FROM description_flags;")
rows = cursor.fetchall()

for item_id, text in rows:
    flags = process_description(text, features)

    for feature, value in flags.items():
        cursor.execute(
            f"UPDATE description_flags SET {feature} = ? WHERE itemId = ?",
            (value, item_id)
        )

conn.commit()
conn.close()


## Формирование правил (CLIPS)

В данном шаге генерируются правила для системы CLIPS на основе данных из базы данных, которая содержит информацию о сочетаниях компонентов косметических средств (из пункта 3). Используем SQL запросы для извлечения информации и формируем соответствующие правила.

- читаем таблицу с данными о сочетаниях компонентов (first_element, second_element), а также результатах их взаимодействия (result) и дополнительными комментариями (comment).

- на основе каждого сочетания компонентов (например, первого и второго элемента) формируется правило для системы CLIPS. В каждом правиле проверяется наличие двух компонентов в списке элементов. Если они присутствуют, то активируется факт с результатом (например, результат взаимодействия этих компонентов).


In [ ]:
import sqlite3
from IPython.display import display, Markdown

# Подключение к SQLite
conn = sqlite3.connect("components-combo-3.db")
cursor = conn.cursor()

# Получаем данные из таблицы
cursor.execute("SELECT first_element, second_element, result, comment FROM components")
rows = cursor.fetchall()
conn.close()

# Генерация правил CLIPS с использованием (or)
clips_rules = []
for i, (first, second, result, comment) in enumerate(rows, 1):
    # Формируем информационное сообщение, если comment не пустой
    info_fact = ""
    if result in ["-1", "0"] or comment:
        info_message = f"Сочетание {first} и {second}: "
        if result == "-1":
            info_message += "Отрицательное взаимодействие"
        elif result == "0":
            info_message += "Есть ограничения по взаимодействию"
        elif result == "1":
            info_message += "Можно сочетать"
        if comment:
            info_message += f", {comment}"
        info_fact = f'(assert (info "{info_message}"))'

    rule = f"""
(defrule правило_{i}
    (or
        (and (first_element "{first}") (second_element "{second}"))
        (and (first_element "{second}") (second_element "{first}"))
    )
    =>
    (assert (result "{result}"))
    {info_fact}
)"""
    clips_rules.append(rule)

# Объединяем все правила
clips_code = "\n".join(clips_rules)

# Красивый вывод с подсветкой синтаксиса
display(Markdown(f"```clips\n{clips_code}\n```"))

# Сохранение в файл
with open("generated_rules_new.clp", "w") as f:
    f.write(clips_code)

print(f"Сгенерировано {len(rows)} правил. Файл: 'generated_rules_new.clp'")


Добавим правила, которые отрабатывают, если был найден компонент из списка запрещенных

In [ ]:
# Список запрещенных компонентов
banned_ingredients = [
    "Benzene",
    "Formaldehyde",
    "Chloroform",
    "Methanol",
    "1,2-Dichloroethane",
    "Nitrosamines",
    "Acetaldehyde",
    "Bis(2-ethylhexyl) phthalate (DEHP)",
    "Dibutyl phthalate (DBP)",
    "Diisobutyl phthalate (DIBP)",
    "Toluene",
    "DDT (Dichlorodiphenyltrichloroethane)",
    "Hexachlorobenzene",
    "Lindane",
    "CI 12150",
    "CI 42555",
    "PABA (Para-aminobenzoic acid)",
    "4-Methylbenzylidene camphor (4-MBC)",
    "Hydroquinone",
    "Cresols",
    "Musk ketone",
    "Musk xylene"
]

clips_rules = []

for i, ingredient in enumerate(banned_ingredients, 1):
    rule = f"""
(defrule banned_ingredient_{i}
    (element "{ingredient}")
    =>
    (assert (result "-1"))
    (assert (info "Внимание! Обнаружен запрещённый компонент: {ingredient}"))
)
"""
    clips_rules.append(rule)

clips_code = "\n".join(clips_rules)

with open("banned_ingredients_rules.clp", "w") as f:
    f.write(clips_code)

print(f"Сгенерировано {len(banned_ingredients)} правил. Файл: 'banned_ingredients_rules.clp'")

Сгенерировано 22 правил. Файл: 'banned_ingredients_rules.clp'


In [ ]:
# после всех правил, которые выдают промежуточные результаты,
# добавляем правила, которые обрабатывают промежуточные и формируют финальный результат

# (defrule set_final_minus_1
#     (declare (salience -10))
#     (result "-1")
#     =>
#     (assert (final_result "-1"))
#     (halt)
# )

# (defrule set_final_0
#     (declare (salience -10))
#     (not (result "-1"))
#     (result "0")
#     =>
#     (assert (final_result "0"))
#     (halt)
# )

# (defrule set_final_1
#     (declare (salience -10))
#     (not (result "-1"))
#     (not (result "0"))
#     (result "1")
#     =>
#     (assert (final_result "1"))
#     (halt)
# )


In [ ]:
# Характеристики из описаний средств, относительно которых будут строится правила
features = [
    "увлажняющий",
    "антивозрастной",
    "выравнивает_тон",
    "питательный",
    "для_чувствительной",
    "для_жирной",
    "от_акне",
    "для_проблемной",
    "для_сухой",
    "для_комбинированной",
    "можно_поверх_макияжа",
    "от_черных_точек",
    "сияние_кожи",

    "регенерирующий",
    "матирующий",
    "от_отеков",
    "для_обезвоженной",
    "для_нормальной",
    "spf_защита",
    "пилинг",
    "энзимный",
    "уменьшает_поры",
    "веганский",
    "гипоаллергенный",
    "некомедогенный",
    "с_маслами",
    "с_экстрактами",
    "минеральный",
    "органический",
    "для_подростков",
    "ночной_уход",
    "дневной_уход",
    "успокаивающий",
    "очищающий",
    "отшелушивающий",
    "противовоспалительный",
    "ароматизированный",
    "неароматизированный",
    "гидрофильный",
    "нежирный",
    "плотный",
    "легкий"
]

In [ ]:
# характеристики пользователя, по которым строятся правила (слева типы фактов, справа чему они могут быть равны)
user_characteristics = {
    "skin_type": ["Сухая", "Жирная", "Комбинированная"],
    "sensitive_skin": ["Да"],
    "tendencies": ["Воспаления", "Акне", "Расширенные_поры", "Шелушения", "Пигментация", "Возрастная_кожа"],
    "age" :["Подростковая", "Молодая", "Зрелая"],
    "had_procedure":["Да", "Нет"],
    "procedure_types":["Инъекции", "Пилинг", "Механическая чистка"],
    "has_allergy":["Да"],
    #"Аллергии_на_компоненты", компоненты будут искаться в составе отдельной функцией, а не правилами

    "weather":["Солнечная", "Тёплая", "Пасмурная", "Холодная"],
    "makeup":["Да", "Нет"]
}


In [ ]:
import sqlite3

conn = sqlite3.connect('product_user_characteristics.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS characteristics_rules (
    product_feature TEXT,
    user_fact TEXT,
    user_feature TEXT,
    info TEXT
)
''')

for feature in features:
    for fact, user_features in user_characteristics.items():
        for user_feature in user_features:
            cursor.execute('''
            INSERT INTO characteristics_rules (product_feature, user_fact, user_feature, info)
            VALUES (?, ?, ?, ?)
            ''', (feature, fact, user_feature, ''))

conn.commit()
conn.close()

print("Таблица успешно создана и заполнена.")


Таблица успешно создана и заполнена.


Заполняем столбец info рекомендациями (если рекомендаций по паре нет, то строку удаляем), добавляем столбец result - окрас рекомендации (1 - положительная, 0 - внимание, -1 - отрицательная).
Добавим автоматическую генераци правил по этой таблице

In [ ]:
import sqlite3
from IPython.display import display, Markdown

conn = sqlite3.connect("product_user_characteristics.db")
cursor = conn.cursor()

cursor.execute("SELECT product_feature, user_fact, user_feature, result, info FROM characteristics_rules")
rows = cursor.fetchall()
conn.close()

clips_rules = []
for i, (product_feat, user_fact, user_feat, result, info) in enumerate(rows, 1):
    info_fact = f'(assert (info "{result}, {info}"))' if info else ""

    rule = f"""
(defrule rule_{i}
    (product_feature "{product_feat}")
    (user_fact {user_fact} "{user_feat}")
    =>
    (assert (result "{result}"))
    {info_fact}
)"""
    clips_rules.append(rule.strip())

clips_code = "\n\n".join(clips_rules)

display(Markdown(f"```clips\n{clips_code}\n```"))

with open("user_product_rules.clp", "w") as f:
    f.write(clips_code)

print(f"Сгенерировано {len(rows)} правил. Файл: 'user_product_rules.clp'")

In [ ]:
# #также добавим пару правил, которые не генерируются автоматически по таблице:
# (defrule new_to_retinol
#     (element "Retinol")
#     =>
#     (assert (info "Если раньше не пользовались ретинолом, начинайте с малого количества и постепенно увеличивайте дозу, чтобы избежать раздражения."))
# )
# (defrule sun-warning
#   (weather "Солнечная")
#   (or
#     (element "Retinol")
#     (element "Blue Retinol Bakuchiol")
#     (element "Retinol Acetate")
#     (element "Retinol Palmitate")
#     (element "Retinal")
#     (element "Retinyl Propionate")
#     (element "Retinol Retinoate")
#     (element "Retinol encapsulated and liposomal form")
#     (element "AHA-acids")
#     (element "Lactic Acid")
#     (element "Glycolic Acid")
#     (element "Mandelic Acid 2–10 %* рН = 3–4")
#     (element "BHA-acids")
#     (element "Salicylic Acid 0.5–2 % pH = 3–4")
#     (element "salicylic acid")
#     (element "salicylic")
#     (element "PHA-acids")
#     (element "Lactobionic Acid")
#     (element "Gluconolactone 2–10 % pH = 3–4")
#     (element "Gluconolactone")
#     (element "Azelaic Acid")
#     (element "Benzoyl Peroxide 2.5–5 %")
#     (element "Benzoyl Peroxide")
#   )
#   =>
#   (assert (info "Поскольку погода солнечная и используются фотосенсибилизирующие компоненты, обязательно применяйте SPF 50+ для защиты кожи."))
# )

